In [1]:
import os
import errno
import argparse
import sys
import pickle

import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model

from data_utils import load_CIFAR_data, load_EMNIST_data, generate_partial_data, generate_bal_private_data
from FedMD import FedMD
from Neural_Networks import train_models, cnn_2layer_fc_model, cnn_3layer_fc_model

2022-10-20 09:52:37.607723: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv('iot23_combined.csv', index_col=0)

/tmp/ipykernel_91755/3197563439.py:1: DtypeWarning: Columns (1,2,3,4,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('iot23_combined.csv', index_col=0)


In [3]:
for label in df.label.unique():

    if 'PartOfAHorizontalPortScan' in label:
        df.loc[(df.label == label), 'label'] = 'PartOfAHorizontalPortScan'

    if 'Benign' in label:
#         print(label)
        df.loc[(df.label == label), 'label'] = 'Benign'

    if 'DDoS' in label:
#         print(label)
        df.loc[(df.label == label), 'label'] = 'DDoS'

    if 'Part' in label:
#         print(label)
        df.loc[(df.label == label), 'label'] = 'PartOfAHorizontalPortScan'

    if 'Scan' in label:
#         print(label)
        df.loc[(df.label == label), 'label'] = 'PartOfAHorizontalPortScan'

    if 'Ma' in label:
        df.loc[(df.label == label), 'label'] = 'PartOfAHorizontalPortScan'

df.loc[(df.label == '(em-'), 'label'] = 'PartOfAHorizontalPortScan'
df.loc[(df.label == '-   Mcan'), 'label'] = 'PartOfAHorizontalPortScan'
df.loc[(df.label == '-  n'), 'label'] = 'PartOfAHorizontalPortScan'

df.loc[(df.label == 'C&C-HeartBeat'), 'label'] = 'C&C'
df.loc[(df.label == 'C&C-FileDownload'), 'label'] = 'C&C'
df.loc[(df.label == 'C&C-Torii'), 'label'] = 'C&C'
df.loc[(df.label == 'C&C-HeartBeat-FileDownload'), 'label'] = 'C&C'
df.loc[(df.label == 'C&C-Mirai'), 'label'] = 'C&C'
df.loc[(df.label == 'FileDownload'), 'label'] = 'C&C'

df.loc[(df.label == '-1'), 'label'] = 'Attack'

In [4]:
df.label.value_counts()

PartOfAHorizontalPortScan    825712
Okiru                        262690
Benign                       197763
DDoS                         138693
C&C                           15521
Attack                         4290
Name: label, dtype: int64

In [5]:
CATEGORICAL_COLUMNS = ['label']

from sklearn.preprocessing import OrdinalEncoder
enc = OrdinalEncoder()
enc.fit(df[CATEGORICAL_COLUMNS])
OrdinalEncoder()
enc.categories_
df[CATEGORICAL_COLUMNS] = enc.transform(df[CATEGORICAL_COLUMNS])
# df = df.reset_index()
# df=df.drop(['index'],axis=1)
df["label"] = df["label"].astype(int)

In [6]:
df.label.value_counts()

5    825712
4    262690
1    197763
3    138693
2     15521
0      4290
Name: label, dtype: int64

In [7]:
df = df.drop(columns=['duration', 'orig_bytes', 'resp_bytes', 'missed_bytes', 
                      'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes'])

In [8]:
X = df.drop(columns=['label']).values
y = df.label.values

In [9]:
from tab2img.converter import Tab2Img

model = Tab2Img()
images = model.fit_transform(X, y)

In [10]:
print(images.shape, y.shape)

(1444669, 9, 9) (1444669,)


In [11]:
images[0]

array([[1., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [12]:
y[0]

5

In [28]:
conf_file = os.path.abspath("conf/EMNIST_balance_conf.json")

In [29]:
with open(conf_file, "r") as f:
    conf_dict = eval(f.read())

    #n_classes = conf_dict["n_classes"]
    model_config = conf_dict["models"]
    pre_train_params = conf_dict["pre_train_params"]
    model_saved_dir = conf_dict["model_saved_dir"]
    model_saved_names = conf_dict["model_saved_names"]
    is_early_stopping = conf_dict["early_stopping"]
    # public_classes = conf_dict["public_classes"]
    # private_classes = conf_dict["private_classes"]
    
    # emnist_data_dir = conf_dict["EMNIST_dir"] 
    N_parties = conf_dict["N_parties"]
    N_samples_per_class = conf_dict["N_samples_per_class"]
    
    N_rounds = conf_dict["N_rounds"]
    N_alignment = conf_dict["N_alignment"]
    N_private_training_round = conf_dict["N_private_training_round"]
    private_training_batchsize = conf_dict["private_training_batchsize"]
    N_logits_matching_round = conf_dict["N_logits_matching_round"]
    logits_matching_batchsize = conf_dict["logits_matching_batchsize"]
    
    result_save_dir = conf_dict['result_save_dir']

del conf_dict, conf_file

In [15]:
private_classes = [5, 1, 2]
public_classes = [0, 3, 4]
n_classes = len(public_classes) + len(private_classes)

In [16]:
from sklearn.model_selection import train_test_split

public_train, private_train, public_target, private_target = train_test_split(images, y, test_size=0.5, random_state=42)

print(public_train.shape, public_target.shape, private_train.shape, private_target.shape)

(722334, 9, 9) (722334,) (722335, 9, 9) (722335,)


In [17]:
X_train_public, X_test_public, y_train_public, y_test_public = train_test_split(public_train, 
                                                                                public_target, 
                                                                                test_size=0.2, 
                                                                                random_state=42)

In [18]:
public_dataset = {"X": X_train_public, "y": y_train_public}

In [19]:
X_train_private, X_test_private, y_train_private, y_test_private = train_test_split(private_train, 
                                                                                private_target, 
                                                                                test_size=0.2, 
                                                                                random_state=42)

In [20]:
# y_train_private += len(public_classes)  # do not know if necessary
# y_test_private += len(public_classes)

In [21]:
private_data, total_private_data \
    = generate_bal_private_data(X_train_private, y_train_private, 
                                N_parties = N_parties,             
                                classes_in_use = private_classes, 
                                N_samples_per_class = N_samples_per_class, 
                                data_overlap = False)

In [22]:
X_tmp, y_tmp = generate_partial_data(X = X_test_private, y= y_test_private, 
                                        class_in_use = private_classes, verbose = True)
private_test_data = {"X": X_tmp, "y": y_tmp}
del X_tmp, y_tmp

X shape : (103933, 9, 9)
y shape : (103933,)


In [23]:
CANDIDATE_MODELS = {"2_layer_CNN": cnn_2layer_fc_model, 
                    "3_layer_CNN": cnn_3layer_fc_model}

In [24]:
parties = []
if model_saved_dir is None:
    for i, item in enumerate(model_config):
        model_name = item["model_type"]
        model_params = item["params"]
        tmp = CANDIDATE_MODELS[model_name](n_classes=n_classes, 
                                            input_shape=(9,9),
                                            **model_params)
        print("model {0} : {1}".format(i, model_saved_names[i]))
        print(tmp.summary())
        parties.append(tmp)
        
        del model_name, model_params, tmp
    #END FOR LOOP
    pre_train_result = train_models(parties, 
                                    X_train_public, y_train_public, 
                                    X_test_public, y_test_public,
                                    save_dir = model_saved_dir, save_names = model_saved_names,
                                    early_stopping = is_early_stopping,
                                    **pre_train_params
                                    )
else:
    dpath = os.path.abspath(model_saved_dir)
    model_names = os.listdir(dpath)
    for name in model_names:
        tmp = None
        tmp = load_model(os.path.join(dpath ,name))
        parties.append(tmp)

model 0 : CNN_128_256
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 9, 9)]            0         
                                                                 
 reshape (Reshape)           (None, 9, 9, 1)           0         
                                                                 
 conv2d (Conv2D)             (None, 9, 9, 128)         1280      
                                                                 
 batch_normalization (BatchN  (None, 9, 9, 128)        512       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 9, 9, 128)         0         
                                                                 
 dropout (Dropout)           (None, 9, 9, 128)         0         
                                       

2022-10-20 09:52:59.658311: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-20 09:52:59.684781: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-20 09:52:59.684953: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-20 09:52:59.685284: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

None
model 3 : CNN_256_256
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 9, 9)]            0         
                                                                 
 reshape_3 (Reshape)         (None, 9, 9, 1)           0         
                                                                 
 conv2d_6 (Conv2D)           (None, 9, 9, 256)         2560      
                                                                 
 batch_normalization_6 (Batc  (None, 9, 9, 256)        1024      
 hNormalization)                                                 
                                                                 
 activation_9 (Activation)   (None, 9, 9, 256)         0         
                                                                 
 dropout_6 (Dropout)         (None, 9, 9, 256)         0         
                                

2022-10-20 09:53:02.150728: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8401
2022-10-20 09:53:03.112434: W tensorflow/stream_executor/gpu/asm_compiler.cc:111] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2022-10-20 09:53:03.114652: W tensorflow/stream_executor/gpu/asm_compiler.cc:230] Falling back to the CUDA driver for PTX compilation; ptxas does not support CC 8.6
2022-10-20 09:53:03.114687: W tensorflow/stream_executor/gpu/asm_compiler.cc:233] Used ptxas at ptxas
2022-10-20 09:53:03.114803: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] UNIMPLEMENTED: ptxas ptxas too old. Falling back to the driver to compile.
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be 

4515/4515 [==============================] - 14s 3ms/step - loss: 0.8716 - accuracy: 0.6929 - val_loss: 0.9605 - val_accuracy: 0.6940
Epoch 2/20
4515/4515 [==============================] - 11s 2ms/step - loss: 0.8638 - accuracy: 0.6938 - val_loss: 0.8619 - val_accuracy: 0.6940
Epoch 3/20
4515/4515 [==============================] - 11s 3ms/step - loss: 0.8632 - accuracy: 0.6938 - val_loss: 0.8602 - val_accuracy: 0.6940
Epoch 4/20
4515/4515 [==============================] - 11s 3ms/step - loss: 0.8629 - accuracy: 0.6938 - val_loss: 0.8606 - val_accuracy: 0.6934
Training model  1
Epoch 1/20
4515/4515 [==============================] - 13s 3ms/step - loss: 0.8730 - accuracy: 0.6928 - val_loss: 1.1489 - val_accuracy: 0.6940
Epoch 2/20
4515/4515 [==============================] - 13s 3ms/step - loss: 0.8638 - accuracy: 0.6937 - val_loss: 0.8634 - val_accuracy: 0.6940
Epoch 3/20
4515/4515 [==============================] - 13s 3ms/step - loss: 0.8631 - accuracy: 0.6938 - val_loss: 0.8640 -

In [25]:
del  X_train_public, y_train_public, X_test_public, y_test_public, \
    X_train_private, y_train_private, X_test_private, y_test_private

In [26]:
fedmd = FedMD(parties, 
                public_dataset = public_dataset,
                private_data = private_data, 
                total_private_data = total_private_data,
                private_test_data = private_test_data,
                N_rounds = N_rounds,
                N_alignment = N_alignment, 
                N_logits_matching_round = N_logits_matching_round,
                logits_matching_batchsize = logits_matching_batchsize, 
                N_private_training_round = N_private_training_round, 
                private_training_batchsize = private_training_batchsize)
    
initialization_result = fedmd.init_result
pooled_train_result = fedmd.pooled_train_result

collaboration_performance = fedmd.collaborative_training()

start model initialization: 
model  0
start full stack training ... 
full stack training done

model  1
start full stack training ... 
full stack training done

model  2
start full stack training ... 
full stack training done

model  3
start full stack training ... 
full stack training done

model  4
start full stack training ... 
full stack training done

model  5
start full stack training ... 
full stack training done

model  6
start full stack training ... 
full stack training done

model  7
start full stack training ... 
full stack training done

model  8
start full stack training ... 
full stack training done

model  9
start full stack training ... 
full stack training done

calculate the theoretical upper bounds for participants: 
the upper bounds are: [0.8516544103622437, 0.8516736626625061, 0.8515582084655762, 0.8466992974281311, 0.8480655550956726, 0.8515582084655762, 0.8516159653663635, 0.8516159653663635, 0.8516159653663635, 0.8515582084655762]
round  0
update logits ... 
te

In [30]:
if result_save_dir is not None:
    save_dir_path = os.path.abspath(result_save_dir)
    #make dir
    try:
        os.makedirs(save_dir_path)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise    


with open(os.path.join(save_dir_path, 'pre_train_result.pkl'), 'wb') as f:
    pickle.dump(pre_train_result, f, protocol=pickle.HIGHEST_PROTOCOL)
with open(os.path.join(save_dir_path, 'init_result.pkl'), 'wb') as f:
    pickle.dump(initialization_result, f, protocol=pickle.HIGHEST_PROTOCOL)
with open(os.path.join(save_dir_path, 'pooled_train_result.pkl'), 'wb') as f:
    pickle.dump(pooled_train_result, f, protocol=pickle.HIGHEST_PROTOCOL)
with open(os.path.join(save_dir_path, 'col_performance.pkl'), 'wb') as f:
    pickle.dump(collaboration_performance, f, protocol=pickle.HIGHEST_PROTOCOL)